# Generative AI Plot / Choice Generation Demo + Script

In [1]:
!pip install together==1.2.0
!pip install wikipedia-api

  Using cached wikipedia_api-0.7.1.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14346 sha256=37b28097df8b363eee55fe041a1dab71acfa0a6ded0b4a2a610a0d4b19d34d64
  Stored in directory: /root/.cache/pip/wheels/4c/96/18/b9201cc3e8b47b02b510460210cfd832ccf10c0c4dd0522962
Successfully built wikipedia-api


In [2]:
import wikipediaapi
import json

## Proof Of Concept

In [6]:
wiki_wiki = wikipediaapi.Wikipedia('MyProjectName (merlin@example.com)', 'en')


def get_plot(title):
    """
    Retrieves the plot summary of the given title from Wikipedia.
    """
    page = wiki_wiki.page(title)
    if not page.exists():
        print(f"Page '{title}' does not exist.")
        return None

    possible_sections = ['Plot', 'Synopsis', 'Summary', 'Plot summary', 'Story']
    plot_section = None
    for section_title in possible_sections:
        plot_section = page.section_by_title(section_title)
        if plot_section:
            break
    if plot_section:
        return plot_section.text
    else:
        print(f"No plot section found for '{title}'.")
        return None

In [10]:
IP = 'Dracula'

system_prompt = f"""
Your job is to adapt the story of {IP} and create a branching storyline that incorporates similar elements while introducing new twists and engaging paths for players.
Instructions:
- Only generate in plain text without formatting.
- Use simple, clear language that is easy to understand and avoids being overly descriptive.
- Ensure each branching storyline offers meaningful player choices and consequences.
- Stay concise, limiting each storyline description to 3-5 sentences.
"""

plot_prompt = f"""
Generate a creative, original adaptation of {IP}'s plot, introducing similar themes and elements while creating 4 new, engaging branching storylines for players.

Output content in the form:
Plot Summary: <PLOT SUMMARY>
Branching Storylines: <BRANCHING STORYLINE DESCRIPTIONS>

{IP} Plot Summary: {get_plot(IP)}"""


In [11]:
plot_prompt

'\nGenerate a creative, original adaptation of Dracula\'s plot, introducing similar themes and elements while creating 4 new, engaging branching storylines for players.\n\nOutput content in the form:\nPlot Summary: <PLOT SUMMARY>\nBranching Storylines: <BRANCHING STORYLINE DESCRIPTIONS>\n\nDracula Plot Summary: Jonathan Harker, a newly qualified English solicitor, visits Count Dracula at his castle in the Carpathian Mountains to help the Count purchase a house near London. Ignoring the Count\'s warning, Harker wanders the castle at night and encounters three vampire women; Dracula rescues Harker, and gives the women a small child bound inside a bag. Harker awakens in bed; soon after, Dracula leaves the castle, abandoning him to the women. Harker escapes and ends up delirious in a Budapest hospital. Dracula takes a ship called the Demeter for England with boxes of earth from his castle. The captain\'s log narrates the crew\'s disappearance until he alone remains, bound to the helm to ma

In [12]:
from together import Together

client = Together(api_key='')

output = client.chat.completions.create(
    model="meta-llama/Llama-3-70b-chat-hf",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": plot_prompt}
    ],
)

In [ ]:
plot_output =output.choices[0].message.content
print(plot_output)

Here is a creative, original adaptation of Dracula's plot with 4 new branching storylines for players:

Plot Summary: In the small town of Ravenswood, a mysterious stranger named Count Draconis arrives, seeking to purchase an old mansion on the outskirts of town. Local solicitor, Emily Welles, is tasked with finalizing the sale, but soon discovers that the Count's true intentions are far more sinister. As the town falls under the Count's dark influence, Emily must join forces with a group of allies to uncover the truth behind the Count's powers and stop him before it's too late.

Branching Storylines:

**Path 1: The Mysterious Heir**
Emily discovers a cryptic letter suggesting that she is the last living heir of an ancient bloodline, making her the only one capable of defeating the Count. As she delves deeper into her family's history, she must choose between embracing her newfound heritage or rejecting it and risking everything. Will she uncover the secrets of her past and claim her b

In [3]:
def generate_continuation(context, previous_scenario=None, previous_choice=None, num_choices=4, choices_left=10):
    """
    Generates a continuation of the story based on the given context and an optional choice.
    choices_left details the amount of urgency left, to account for story pacing.
    """
    system_prompt = f"""
    Your job is to tell a continuation of the story based on what previously happened.
    You will be given the overall plot synopsis to help guide the story in that direction.
    The player has {choices_left} choice(s) left.

    Instructions:
    - Use simple, clear language that is easy to understand and avoids being overly descriptive.
    - Stay in third-person point of view.
    - Ensure each branching storyline offers meaningful player choices and consequences.
    - Stay concise, limiting each storyline description to 3-5 sentences.
    - The player has around 10 choices in total. If they have 5-7 left, ramp up the story, aiming for 3-4 choices left as the climax. Begin to aim for an interesting conclusion at 2-1 choices left.
    - All outputs should be in JSON format like so. Do not include anything else:
    {{
      "story_continuation": "[Your story continuation here]",
      "choices": [
        {{
          "choice": 1,
          "action": "[First possible action]"
        }},
        {{
          "choice": 2,
          "action": "[Second possible action]"
        }}
        ...
      ]
    }}
    """

    if previous_choice:
        prompt = f"""
        Write a continuation of the story that progresses the story forward. Include {num_choices} choices for the player to choose from given this generated situation.
        All outputs should be in JSON format. Do not include anything else.

        The following scenario has just occured:
        {previous_scenario}

        The player made the following choice:
        {previous_choice}

        Plot Summary: {context}
        """
    else:
        prompt = f"""
        We are at the start of our adventure. Write a continuation of the story that progresses the story forward. Include {num_choices} choices for the player to choose from given this generated situation.
        All outputs should be in JSON format.

        Plot Summary: {context}
        """
    try:

      output = client.chat.completions.create(
      model="meta-llama/Llama-3-70b-chat-hf",
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": prompt}
      ],
    )
      return output.choices[0].message.content
    except Exception as e:
        print(f"Error during generation: {e}")
        return None


In [ ]:
# Example:
# Generated Plot Summary now becomes context: In the small town of Ravenswood, a mysterious stranger named Count Draconis arrives, seeking to purchase an old mansion on the outskirts of town. Local solicitor, Emily Welles, is tasked with facilitating the sale, but soon discovers that the Count's true intentions are far more sinister. As the town falls under the Count's dark influence, Emily must join forces with a group of allies to uncover the truth behind the Count's powers and stop him before it's too late.
# Each path is added to the context (can also put the path name on screen, which I think is cool and engaging): e.g. Emily discovers that she is the last living heir of the original owner of the mansion, and that the Count's true intention is to claim the estate's dark legacy for himself. Players must navigate the complex web of family secrets and ancient curses to uncover the truth about their own past and the source of the Count's power.

In [ ]:
example_context = "In the small town of Ravenswood, a mysterious stranger named Count Draconis arrives, seeking to purchase an old mansion on the outskirts of town. Local solicitor, Emily Welles, is tasked with facilitating the sale, but soon discovers that the Count's true intentions are far more sinister. As the town falls under the Count's dark influence, Emily must join forces with a group of allies to uncover the truth behind the Count's powers and stop him before it's too late. Emily discovers that she is the last living heir of the original owner of the mansion, and that the Count's true intention is to claim the estate's dark legacy for himself. Players must navigate the complex web of family secrets and ancient curses to uncover the truth about their own past and the source of the Count's power."

output = generate_continuation(example_context, choices_left=10)
output

'{\n  "story_continuation": "Emily stood outside the old mansion, her heart racing as she watched Count Draconis\'s carriage disappear into the night. She couldn\'t shake the feeling that she had just made a terrible mistake. As she turned to leave, she noticed a piece of paper on the ground, blown by the wind. It was a letter, addressed to her, with no indication of who had written it. The words sent a chill down her spine: \'You are the last living heir of Malcolm Welles, the original owner of the mansion. The Count\'s true intention is to claim the estate\'s dark legacy for himself. Meet me at the old windmill on the outskirts of town at midnight if you want to know the truth.\'",\n  "choices": [\n    {\n      "choice": 1,\n      "action": "Go to the windmill at midnight, alone and unprepared"\n    },\n    {\n      "choice": 2,\n      "action": " Ignore the letter and try to forget about the whole thing"\n    },\n    {\n      "choice": 3,\n      "action": "Bring a group of trusted f

In [ ]:
parsed_data = json.loads(output)
cleaned_json = json.dumps(parsed_data, indent=4)
print(cleaned_json)

{
    "story_continuation": "Emily stood outside the old mansion, her heart racing as she watched Count Draconis's carriage disappear into the night. She couldn't shake the feeling that she had just made a terrible mistake. As she turned to leave, she noticed a piece of paper on the ground, blown by the wind. It was a letter, addressed to her, with no indication of who had written it. The words sent a chill down her spine: 'You are the last living heir of Malcolm Welles, the original owner of the mansion. The Count's true intention is to claim the estate's dark legacy for himself. Meet me at the old windmill on the outskirts of town at midnight if you want to know the truth.'",
    "choices": [
        {
            "choice": 1,
            "action": "Go to the windmill at midnight, alone and unprepared"
        },
        {
            "choice": 2,
            "action": " Ignore the letter and try to forget about the whole thing"
        },
        {
            "choice": 3,
         

In [ ]:
output = generate_continuation(example_context,
                               previous_scenario=parsed_data.get("story_continuation"),
                               previous_choice=parsed_data.get("choices")[0]['action'],
                               choices_left=9)
output

'{\n  "story_continuation": "Emily arrived at the old windmill on the outskirts of town, the creaking of the wooden blades echoing through the darkness. She saw a figure cloaked in shadows, waiting for her. As she approached, the figure stepped forward, revealing a woman with piercing green eyes. \'I\'m Arabella, a member of a secret society that has been watching the Count\'s movements,\' she said. \'We know the truth about your family\'s past and the dark legacy that comes with it. But we must be quick, the Count\'s minions are closing in.\'",\n  "choices": [\n    {\n      "choice": 1,\n      "action": "Ask Arabella about the secret society and what they know about the Count\'s powers"\n    },\n    {\n      "choice": 2,\n      "action": "Demand to know more about Emily\'s family\'s past and the dark legacy"\n    },\n    {\n      "choice": 3,\n      "action": "Refuse to trust Arabella and try to leave"\n    },\n    {\n      "choice": 4,\n      "action": "Ask Arabella if she knows anyt

In [ ]:
parsed_data = json.loads(output)
cleaned_json = json.dumps(parsed_data, indent=4)
print(cleaned_json)

{
    "story_continuation": "Emily arrived at the old windmill on the outskirts of town, the creaking of the wooden blades echoing through the darkness. She saw a figure cloaked in shadows, waiting for her. As she approached, the figure stepped forward, revealing a woman with piercing green eyes. 'I'm Arabella, a member of a secret society that has been watching the Count's movements,' she said. 'We know the truth about your family's past and the dark legacy that comes with it. But we must be quick, the Count's minions are closing in.'",
    "choices": [
        {
            "choice": 1,
            "action": "Ask Arabella about the secret society and what they know about the Count's powers"
        },
        {
            "choice": 2,
            "action": "Demand to know more about Emily's family's past and the dark legacy"
        },
        {
            "choice": 3,
            "action": "Refuse to trust Arabella and try to leave"
        },
        {
            "choice": 4,
  

## Recursive Script

In [19]:
all_jsons = {}
GENERATION_THRESHOLD = 8
MAX_RETRIES = 5


def initialize_story(IP, num_choices=4):
    """
    Generates the initial story framework for an interactive narrative based on a specific intellectual property (IP).
    This function adapts the original plot of the IP to create a new branching storyline with similar themes but fresh twists and engaging paths.

    Parameters:
    - `IP` (str): The intellectual property (e.g., movie, book, or game) to base the story on.
    - `num_choices` (int): The number of branching story paths to create. Default is 4.
    """

    system_prompt = f"""
    Your job is to adapt the story of {IP} and create a branching storyline that incorporates similar elements while introducing new twists and engaging paths for players.
    Instructions:
    - Use simple, clear language that is easy to understand and avoids being overly descriptive.
    - Ensure each branching storyline offers meaningful player choices and consequences.
    - Stay concise, limiting each storyline description to 3-5 sentences.
    - All outputs should be in JSON format like so. Do not include anything else:
    {{
      "plot_summary": "[Your initial plot summary here]",
      "branching_storylines": [
        {{
          "path": 1,
          "path_name": "Title of path"
          "story_line": "[Summary of first storyline]"
        }},
        {{
          "path": 2,
          "path_name": "Title of path"
          "story_line": "[Summary of second storyline]"
        }}
        ...
      ]
    }}
    """

    plot_prompt = f"""
    Generate a creative, original adaptation of {IP}'s plot, introducing similar themes and elements while creating 4 new, engaging branching storylines for players.
    All outputs should be in JSON format. Do not include anything else.

    {IP} Plot Summary: {get_plot(IP)}"""

    output = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": plot_prompt}
        ],
    )

    return output.choices[0].message.content


def assert_valid_json(json_str):
    """
    Attempts to parse a JSON string with retries upon failure.

    Parameters:
        json_str (str): The JSON string to parse.
        max_retries (int): Maximum number of retry attempts.

    Returns:
        True if valid JSON, False otherwise.
    """
    try:
        parsed_data = json.loads(json_str)
        return True
    except json.JSONDecodeError as e:
        return False


# Recursive Builder:
def populate_JSON(context,
                  previous_scenario, previous_choice,
                  label_idx,
                  num_choices=4, choices_left=10):
    """
    Recursively populates a JSON structure for a branching storyline.

    Parameters:
    - `context` (str): The overall storyline context to guide the creation of new continuations.
    - `previous_scenario` (str): The preceding storyline scenario from which new continuations branch.
    - `previous_choice` (str): The player's previous choice that led to the current scenario.
    - 'label_idx' (str): Label for generated choice, used to id/navigate all_jsons.
    - `num_choices` (int): The number of branching choices to create at each step. Default is 4.
    - `choices_left` (int): The remaining depth of choices before the story concludes.
    """

    if choices_left <= GENERATION_THRESHOLD:
      return

    # Assert generated JSON is validly formatted given MAX_RETRIES
    for attempt in range(1, MAX_RETRIES + 1):
        output = generate_continuation(
            context,
            previous_scenario,
            previous_choice,
            num_choices=num_choices,
            choices_left=choices_left
        )

        if assert_valid_json(output):
          break
        elif attempt == MAX_RETRIES:
          print(f"Failed to parse JSON after {MAX_RETRIES} attempts.")
          return

    parsed_data = json.loads(output)
    all_jsons[label_idx] = parsed_data

    current_scenario = parsed_data.get("story_continuation")
    for id, choice in enumerate(parsed_data.get("choices")):
        current_choice = choice["action"]
        populate_JSON(context,
                      previous_scenario=current_scenario,
                      previous_choice=current_choice,
                      label_idx = label_idx + str(id + 1), # 1-indexed to match choice idxs
                      num_choices=num_choices,
                      choices_left=choices_left-1)


def create_story(IP, num_choices=4, choices_left=10):
    """
    Creates a branching interactive story based on a specific intellectual property (IP), generating storylines recursively.

    Parameters:
    - `IP` (str): The intellectual property (e.g., movie, book, or game) to base the story on.
    - `num_choices` (int): The number of branching choices to create at each stage of the storyline. Default is 4.
    - `choices_left` (int): The total depth of choices before the story concludes.
    """
    # Assert generated JSON is validly formatted given MAX_RETRIES
    for attempt in range(1, MAX_RETRIES + 1):
        story = initialize_story(IP, num_choices)
        if assert_valid_json(story):
          break
        elif attempt == MAX_RETRIES:
          print(f"Failed to parse JSON after {MAX_RETRIES} attempts.")
          return

    story = json.loads(story)
    all_jsons[""] = story

    initial_context = story.get("plot_summary")

    for id, branch in enumerate(story.get("branching_storylines")):
        path_name = branch["path_name"] # TODO: add to final JSON + Unity
        story_line = branch["story_line"]

        # context for each branch is initial context + overall plot of specific branch
        context = initial_context + " " + story_line
        populate_JSON(context,
                      previous_scenario=None,
                      previous_choice=None,
                      label_idx = str(id + 1), # 1-indexed to match choice idxs
                      num_choices=num_choices,
                      choices_left=choices_left)

In [20]:
create_story('Dracula')

In [21]:
cleaned_json = json.dumps(all_jsons, indent=4)
print(cleaned_json)

{
    "": {
        "plot_summary": "In the modern city of New Haven, a mysterious corporation called 'Eclipse' has taken over the city's infrastructure, and people are disappearing at an alarming rate. The main character, a brilliant journalist named Maya, receives a cryptic message from an anonymous source claiming to have information about Eclipse's true intentions. As Maya delves deeper into the mystery, she discovers that Eclipse is led by a charismatic and enigmatic figure known only as 'The Archon', who seems to have supernatural abilities.",
        "branching_storylines": [
            {
                "path": 1,
                "path_name": "The Whistleblower's Trail",
                "story_line": "Maya follows the trail of clues left by the anonymous source, leading her to a hidden underground bunker where she finds evidence of Eclipse's sinister experiments. She must decide whether to go public with the information or continue investigating to uncover more secrets."
     

In [22]:
with open("example_data.json", "w") as f:
    json.dump(all_jsons, f)